In [2]:
import random
import time
import numpy as np
import pickle
from scipy.io import loadmat
import pandas as pd
import os

def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

def get_csi_amplitudes(csi_value):
    csi_amplitudes = []
    for v in csi_value:
        amplitude = np.absolute(v)
        csi_amplitudes.append(amplitude)
    return csi_amplitudes

def get_data(mat_filename, transmitter, receiver, authenticated_pair = [' ',' ']):
    annots = loadmat(mat_filename)
    data = []
    for i in range(len(annots['Raw_Cell_Matrix'])):
        authenticated = 0
        if transmitter == authenticated_pair[0] and receiver == authenticated_pair[1]:
            authenticated = 1
        data.append([
            int(annots['Raw_Cell_Matrix'][i][0]['timestamp_low'][0][0][0][0]),
            transmitter,
            receiver,
            annots['Raw_Cell_Matrix'][i][0]['agc'][0][0][0][0]
            ] + get_csi_amplitudes(annots['Raw_Cell_Matrix'][i][0]['CSI'][0][0][0][0])
            + [authenticated]
            )

    csi_columns = ['A'+ str(i) for i in range(1,31)]
    columns = ['timestamp', 'transmitter', 'receiver', 'RSS'] + csi_columns + ['Authenticated']
    final_data = pd.DataFrame(data, columns=columns)
    return final_data

# print(prepare_data('data.mat','S13','S21'))

from pathlib import Path

def prepare_data(directory, authenticated_pair):
    csi_columns = ['A'+ str(i) for i in range(1,31)]
    columns = ['timestamp', 'transmitter', 'receiver', 'RSS'] + csi_columns
    all_data = pd.DataFrame([], columns=columns)
    for p in getListOfFiles(Path(directory)):
        pair = p.split('/')[1]
        print(p, pair.split('_')[0], pair.split('_')[1])
        data = get_data(p, pair.split('_')[0], pair.split('_')[1], authenticated_pair = authenticated_pair)
        all_data = pd.concat([all_data, data])
    return all_data

In [3]:
authenticated_pair = ['S13', 'S21']
test_data = prepare_data('./test_data',authenticated_pair = authenticated_pair)
df_test = test_data.drop(['timestamp','transmitter', 'receiver','Authenticated'],axis = 1)
dfy_test = test_data[['Authenticated']]

test_data/S19_S11/I12/S19_S11_I12_T4.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T3.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T8.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T6.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T7.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T9.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T10.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T5.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T2.mat S19 S11
test_data/S19_S11/I12/S19_S11_I12_T1.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T5.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T9.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T7.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T1.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T8.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T2.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T4.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T6.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T3.mat S19 S11
test_data/S19_S11/I10/S19_S11_I10_T10.mat S19 S11
test_data/S19_S11/

In [4]:
X_test = df_test
Y_test = dfy_test
Y_test.head()

,Authenticated
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


-------------------------------------RANDOM FOREST CLASSIFIER-------------------------------------

In [6]:
filename = 'rf_model_class.sav'

rf = pickle.load(open(filename, 'rb'))

Accuracy

In [7]:
rf.score(X_test, Y_test)

0.9658496732026144

In [7]:
y_pred = rf.predict(X_test)

Confusion Matrix

In [8]:
from sklearn.metrics import confusion_matrix

y_pred = rf.predict(X_test)

confusion_matrix(Y_test, y_pred)

array([[50433,   979],
       [ 2574, 50054]])

Precision

In [9]:
from sklearn.metrics import precision_score

y_pred = rf.predict(X_test)

precision_score(Y_test, y_pred, average='weighted')

0.9663000255110833

F1 Score

In [10]:
from sklearn.metrics import f1_score

y_pred = rf.predict(X_test)

f1_score(Y_test, y_pred, average='weighted')

0.9658477659868073

-------------------------------------KNN CLASSIFIER-------------------------------------

In [6]:
filename = 'knn_model_class.sav'

knn = pickle.load(open(filename, 'rb'))

In [13]:
knn.score(X_test, Y_test)

KeyboardInterrupt: 

In [14]:
y_pred = knn.predict(X_test)

Confusion Matrix

In [15]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, y_pred)

array([[50471,   941],
       [ 3116, 49512]])

Precision

In [16]:
from sklearn.metrics import precision_score

precision_score(Y_test, y_pred, average='weighted')

0.9618310925247073

F1 Score

In [17]:
from sklearn.metrics import f1_score

f1_score(Y_test, y_pred, average='weighted')

0.9609978677223345

-------------------------------------Logistic Regression-------------------------------------

In [12]:
filename = 'lr_model_class.sav'

lr = pickle.load(open(filename, 'rb'))

Accuracy

In [13]:
lr.score(X_test, Y_test)

0.971482122260669

In [14]:
y_pred = lr.predict(X_test)

Confusion Matrix

In [15]:
from sklearn.metrics import confusion_matrix

y_pred = lr.predict(X_test)

confusion_matrix(Y_test, y_pred)

array([[50319,  1093],
       [ 1874, 50754]])

Precision

In [16]:
from sklearn.metrics import precision_score

y_pred = lr.predict(X_test)

precision_score(Y_test, y_pred, average='weighted')

0.9715934015465552

F1 Score

In [17]:
from sklearn.metrics import f1_score

y_pred = lr.predict(X_test)

f1_score(Y_test, y_pred, average='weighted')

0.9714830173459155

-------------------------------------Support Vector Machine-------------------------------------

In [6]:
filename = 'svm_model_class.sav'

svm = pickle.load(open(filename, 'rb'))

Accuracy

In [7]:
svm.score(X_test, Y_test)

0.9710207612456747

In [14]:
y_pred = svm.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, y_pred)

array([[50318,  1094],
       [ 1921, 50707]])

Precision

In [16]:
from sklearn.metrics import precision_score

precision_score(Y_test, y_pred, average='weighted')

0.971145192859383

F1 Score

In [17]:
from sklearn.metrics import f1_score

f1_score(Y_test, y_pred, average='weighted')

0.9710216225318339